In [114]:
import pandas as pd
import re

In [115]:
pod_name = 'EU16'
total_time = 10 * 60  # seconds

In [116]:
cpu_time = pd.read_csv('bajaj_4_2.csv')
cpu_time.head()

,_time,host,totalCPUTime
0,2019-04-02T04:00:00.000+0000,eu16-app1-1-fra.ops.sfdc.net,507122
1,2019-04-02T04:00:00.000+0000,eu16-app1-10-fra.ops.sfdc.net,1274076
2,2019-04-02T04:00:00.000+0000,eu16-app1-11-fra.ops.sfdc.net,541366
3,2019-04-02T04:00:00.000+0000,eu16-app1-12-fra.ops.sfdc.net,1473492
4,2019-04-02T04:00:00.000+0000,eu16-app1-13-fra.ops.sfdc.net,1482424


In [117]:
hardware = pd.read_csv('hardwarereport.csv')
hardware = hardware[hardware['Pod'] == pod_name]
hardware['host'] = hardware['Host Name'].apply(lambda x: x + '.ops.sfdc.net')
hardware = hardware[['host','CPU_Model' ,'# of CPU Cores']]
hardware.columns = ['host','CPU_Model' ,'cores']
hardware.head()

,host,CPU_Model,cores
220,eu16-db1-12-fra.ops.sfdc.net,Intel(R) Xeon(R) Gold 6150 CPU @ 2.70GHz Stepp...,36.0
1789,eu16-db1-10-fra.ops.sfdc.net,Intel(R) Xeon(R) Gold 6150 CPU @ 2.70GHz Stepp...,36.0
1790,eu16-db1-8-fra.ops.sfdc.net,Intel(R) Xeon(R) Gold 6150 CPU @ 2.70GHz Stepp...,36.0
1791,eu16-db1-6-fra.ops.sfdc.net,Intel(R) Xeon(R) Gold 6150 CPU @ 2.70GHz Stepp...,36.0
1792,eu16-db1-4-fra.ops.sfdc.net,Intel(R) Xeon(R) Gold 6150 CPU @ 2.70GHz Stepp...,36.0


In [118]:
#cpu_time[cpu_time['host'] == 'na40-app1-10-dfw.ops.sfdc.net']

In [119]:
#hardware[hardware['host'].str.contains("eu16")]

In [120]:
data = cpu_time.join(hardware.set_index('host'), on='host', how='left')
data.head()

,_time,host,totalCPUTime,CPU_Model,cores
0,2019-04-02T04:00:00.000+0000,eu16-app1-1-fra.ops.sfdc.net,507122,Intel(R) Xeon(R) Gold 5118 CPU @ 2.30GHz Stepp...,24.0
1,2019-04-02T04:00:00.000+0000,eu16-app1-10-fra.ops.sfdc.net,1274076,Intel(R) Xeon(R) Gold 5118 CPU @ 2.30GHz Stepp...,24.0
2,2019-04-02T04:00:00.000+0000,eu16-app1-11-fra.ops.sfdc.net,541366,Intel(R) Xeon(R) Gold 5118 CPU @ 2.30GHz Stepp...,24.0
3,2019-04-02T04:00:00.000+0000,eu16-app1-12-fra.ops.sfdc.net,1473492,Intel(R) Xeon(R) Gold 5118 CPU @ 2.30GHz Stepp...,24.0
4,2019-04-02T04:00:00.000+0000,eu16-app1-13-fra.ops.sfdc.net,1482424,Intel(R) Xeon(R) Gold 5118 CPU @ 2.30GHz Stepp...,24.0


In [121]:
#cores = hardware[['CPU_Model' ,'cores']].drop_duplicates()
#print cores

In [122]:
def get_clock_speed(CPU_Model):
    clock_speed = re.search('@(.+)GHz', CPU_Model).group(1)
    return float(clock_speed)*1e9
data = data.dropna() # There is some hardware we don't have its 'cores'
data['clock_speed'] = data['CPU_Model'].apply(get_clock_speed)
data['cpu_usage'] = data['totalCPUTime'] * 100 / (data['clock_speed'] * data['cores'] * total_time)
data.head()

,_time,host,totalCPUTime,CPU_Model,cores,clock_speed,cpu_usage
0,2019-04-02T04:00:00.000+0000,eu16-app1-1-fra.ops.sfdc.net,507122,Intel(R) Xeon(R) Gold 5118 CPU @ 2.30GHz Stepp...,24.0,2.300000e+09,0.000002
1,2019-04-02T04:00:00.000+0000,eu16-app1-10-fra.ops.sfdc.net,1274076,Intel(R) Xeon(R) Gold 5118 CPU @ 2.30GHz Stepp...,24.0,2.300000e+09,0.000004
2,2019-04-02T04:00:00.000+0000,eu16-app1-11-fra.ops.sfdc.net,541366,Intel(R) Xeon(R) Gold 5118 CPU @ 2.30GHz Stepp...,24.0,2.300000e+09,0.000002
3,2019-04-02T04:00:00.000+0000,eu16-app1-12-fra.ops.sfdc.net,1473492,Intel(R) Xeon(R) Gold 5118 CPU @ 2.30GHz Stepp...,24.0,2.300000e+09,0.000004
4,2019-04-02T04:00:00.000+0000,eu16-app1-13-fra.ops.sfdc.net,1482424,Intel(R) Xeon(R) Gold 5118 CPU @ 2.30GHz Stepp...,24.0,2.300000e+09,0.000004


In [123]:
data[data.isnull().any(axis=1)]

,_time,host,totalCPUTime,CPU_Model,cores,clock_speed,cpu_usage
